# GenAI course - Evaluation


## Use pre-trained model

Select a specific task involving image or text generation: image transformation, translation, summarisation, Q&A.

Find a dataset with annotated data corresponding to your selected task and load it.

Divide the dataset into a train and a test set.

Find a pre-trained model on the hugging face hub suitable for the selected task and load it.

Select the appropriate metrics to evaluate the considered task.

Use the selected metrics to evalute the model on your test set.

Comment the results.

## Model fine-tuning

Use the train set to fine-tune the pre-trained model.

Use the selected metrics to evalute the fine-tuned model on your test set.

Comment the results.

## To go further

Explore how to implement a model from scrath and train it on your train set.

Use the selected metrics to evalute the model trained from scratch on your test set.

Comment the results.